In [1]:
pip install tf2crf

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from datasets import load_dataset, concatenate_datasets
from sklearn.metrics import classification_report
from tf2crf import CRF, ModelWithCRFLoss

In [3]:
hiner = load_dataset('cfilt/HiNER-original')
train_data = hiner['train']
val_data = hiner['validation']
test_data = hiner['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset hi_ner_config downloaded and prepared to /root/.cache/huggingface/datasets/cfilt___hi_ner_config/HiNER/0.0.2/c2bf095b51bde10ac392c9203c0fcdd1d7c47d2b03b6b455bf277f1afd7feed0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
combined_data = concatenate_datasets([train_data, val_data])

In [5]:
tokens = combined_data['tokens']
ner_tags = combined_data['ner_tags']

In [6]:
all_tokens = [token for sublist in tokens for token in sublist]
all_tags = [tag for sublist in ner_tags for tag in sublist]
token_vocab = sorted(set(all_tokens))
tag_vocab = sorted(set(all_tags))
# tag_vocab.append(int(23))

In [7]:
token_to_id = {token: i for i, token in enumerate(token_vocab)}
tag_to_id = {tag: i for i, tag in enumerate(tag_vocab)}

In [8]:
# Preprocess the training data
train_tokens = train_data['tokens']
train_tags = train_data['ner_tags']
train_tokens_ids = [[token_to_id[token] for token in sublist] for sublist in train_tokens]
train_tags_ids = [[tag_to_id[tag] for tag in sublist] for sublist in train_tags]
max_length = max(len(seq) for seq in train_tokens_ids)  # Calculate the maximum length
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=max_length, padding='post')
train_tags_ids = pad_sequences(train_tags_ids, maxlen=max_length, padding='post')
# train_tags_ids = to_categorical(train_tags_ids, num_classes=len(tag_vocab))

In [9]:
# Preprocess the validation data
val_tokens = val_data['tokens']
val_tags = val_data['ner_tags']
val_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 22)) for token in sublist] for sublist in val_tokens]
val_tags_ids = [[tag_to_id.get(tag, tag_to_id.get('<unk>', 22)) for tag in sublist] for sublist in val_tags]
val_tokens_ids = pad_sequences(val_tokens_ids, maxlen=max_length, padding='post')
val_tags_ids = pad_sequences(val_tags_ids, maxlen=max_length, padding='post')
# val_tags_ids = to_categorical(val_tags_ids, num_classes=len(tag_vocab))

In [10]:
# Preprocess the test data
test_tokens = test_data['tokens']
test_tags = test_data['ner_tags']
test_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 22)) for token in sublist] for sublist in test_tokens]
test_tags_ids = [[tag_to_id.get(tag, tag_to_id.get('<unk>', 22)) for tag in sublist] for sublist in test_tags]
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=max_length, padding='post')
test_tags_ids = pad_sequences(test_tags_ids, maxlen=max_length, padding='post')
# test_tags_ids = to_categorical(test_tags_ids, num_classes=len(tag_vocab))

In [11]:
word_embedding = layers.Embedding(input_dim=len(token_vocab), output_dim=64)
bilstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True))
bilstm_2 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))
dense = layers.Dense(len(tag_vocab))
crf = CRF(dtype=tf.float32)

inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = word_embedding(inputs)
x = bilstm(x)
x = bilstm_2(x)
x = dense(x)
outputs = crf(x)

base_model = tf.keras.Model(inputs, outputs)

model = ModelWithCRFLoss(base_model)

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [13]:
model.fit(train_tokens_ids, train_tags_ids, validation_data=(val_tokens_ids, val_tags_ids), epochs=3, batch_size=128)

Epoch 1/3
593/593 [==============================] - 808s 1s/step - loss: 20.9893 - accuracy: 0.9731 - val_loss_val: 6.3224 - val_val_accuracy: 0.9900
Epoch 2/3
593/593 [==============================] - 788s 1s/step - loss: 4.5046 - accuracy: 0.9927 - val_loss_val: 3.5912 - val_val_accuracy: 0.9943
Epoch 3/3
593/593 [==============================] - 790s 1s/step - loss: 2.5434 - accuracy: 0.9958 - val_loss_val: 2.7997 - val_val_accuracy: 0.9953


In [14]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_tokens_ids, test_tags_ids)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)
print('Test Precision:', test_precision)
print('Test Recall:', test_recall)

677/677 [==============================] - 138s 204ms/step - loss_val: 3.3309 - val_accuracy: 0.9949
Test Loss: 3.3592047691345215
Test Accuracy: 0.9949097633361816


In [14]:
model.save('NLP_NER_BiLSTM-CRF',save_format="tf")

In [17]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions on test data
predictions = model.predict(test_tokens_ids)

# Convert predictions from categorical to label encoded
pred_tags_ids = np.argmax(predictions, axis=-1)

# Convert test tags from one-hot encoded to label encoded
true_tags_ids = np.argmax(test_tags_ids, axis=-1)

# Flatten both arrays
pred_tags_ids_flat = pred_tags_ids.flatten()
true_tags_ids_flat = true_tags_ids.flatten()

# Use inverse_transform to map predicted and true labels back to their original form.
# You can skip this step if you want the classification report for the encoded labels.
pred_tags = [tag_vocab[i] for i in pred_tags_ids_flat]
true_tags = [tag_vocab[i] for i in true_tags_ids_flat]

# Generate classification report
report = classification_report(true_tags, pred_tags, output_dict=True)

report = classification_report(true_tags, pred_tags)
print(report)

677/677 [==============================] - 123s 179ms/step
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      9627
           1       0.92      0.87      0.90      4822
           2       0.94      0.90      0.92      5358
           3       0.91      0.72      0.81      1277
           4       0.79      0.54      0.64       353
           5       0.80      0.52      0.63       137
           6       0.61      0.43      0.51        46
           7       0.60      0.33      0.43        18
           8       1.00      0.44      0.62         9
           9       1.00      0.75      0.86         4
          10       1.00      0.50      0.67         4
          11       0.00      0.00      0.00         2

    accuracy                           0.90     21657
   macro avg       0.79      0.58      0.66     21657
weighted avg       0.90      0.90      0.90     21657



In [16]:
!zip -r NLP_NER_BiLSTM-CRF NLP_NER_BiLSTM-CRF/

  adding: NLP_NER_BiLSTM-CRF/ (stored 0%)
  adding: NLP_NER_BiLSTM-CRF/assets/ (stored 0%)
  adding: NLP_NER_BiLSTM-CRF/saved_model.pb (deflated 90%)
  adding: NLP_NER_BiLSTM-CRF/variables/ (stored 0%)
  adding: NLP_NER_BiLSTM-CRF/variables/variables.index (deflated 66%)
  adding: NLP_NER_BiLSTM-CRF/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: NLP_NER_BiLSTM-CRF/fingerprint.pb (stored 0%)
  adding: NLP_NER_BiLSTM-CRF/keras_metadata.pb (deflated 92%)
